## Ergotropy

The goal of this notebook is to compute the ergotropy $W$ of a system $S$, given an arbitrary initial state and hamiltonian $H$. The ergotropy $W$ is none other that the maximum amount of work extractable from a finite system, and it is given by

$$
W = E(\rho_0) - E(\rho(\tau)) = tr[\rho_0 H] - tr[\rho(\tau) H] = \sum_{j,k}r_j\epsilon_k(|\langle r_j|\epsilon_k \rangle|^2-\delta_{jk})
$$

where we wrote 

$$
\rho_0 = \sum_{j \geqslant 1}r_j|r_j\rangle \langle r_j| \\
H = \sum_{k \geqslant 1}\epsilon_k|\epsilon_k\rangle \langle \epsilon_k| \\
\rho(\tau) = \sum_{j}r_j|\epsilon_j\rangle \langle \epsilon_j|
$$ 

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image
from qutip import (Qobj, about, basis, coherent, coherent_dm, create, destroy,
                   expect, fock, fock_dm, mesolve, qeye, sigmax, sigmay,
                   sigmaz, tensor, thermal_dm)

%matplotlib inline

In [3]:
def kronecker_delta(a,b):
    if a==b: return 1
    else: return 0

In [4]:
def ergotropy(_init_state, _H):
    # for now I assume that init_state is given as a density matrix
    # TODO: add a check to enforce this!
    
    # Calculate eigenvalues and eigenstates
    _init_state_eigens = _init_state.eigenstates()
    _H_eigens = _H.eigenstates()
    
    # Next, I have to order the eigenvalues, decreasing for init_state and ascending for H
    _init_state_eigenvalues_ordered = -np.sort(-_init_state_eigens[0], kind='mergesort') # TODO: do this in a better way
    _H_eigenenergies_ordered = np.sort(_H_eigens[0], kind='mergesort')
    
    _ergotropy = 0
    for j in range(len(_init_state_eigenvalues_ordered)):
        for k in range(len(_H_eigenenergies_ordered)):
            _ergotropy += _init_state_eigenvalues_ordered[j]*_H_eigenenergies_ordered[k] * (
                          abs(_H_eigens[1][k].overlap(_init_state_eigens[1][j]))**2-kronecker_delta(j,k))
    return _ergotropy

In [9]:
state = coherent_dm(N=2, alpha=1.0)
state

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.29192658 0.45464871]
 [0.45464871 0.70807342]]

In [31]:
len(state.eigenenergies())

2

In [14]:
-np.sort(-state.eigenenergies(), kind='mergesort')

array([1., 0.])

In [20]:
state.eigenstates()[1]

array([Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
       Qobj data =
       [[-0.84147098]
        [ 0.54030231]]                                              ,
       Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
       Qobj data =
       [[0.54030231]
        [0.84147098]]                                               ],
      dtype=object)

In [8]:
H = 1.0 * sigmaz() + 0.1 * sigmay()
H

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 1.+0.j   0.-0.1j]
 [ 0.+0.1j -1.+0.j ]]

In [15]:
np.sort(H.eigenenergies(), kind='mergesort')

array([-1.00498756,  1.00498756])

In [29]:
abs(H.eigenstates()[1][0].overlap(state.eigenstates()[1][0]))**2

0.29295921052367796

In [44]:
ergotropy(coherent_dm(N=2, alpha=1.0),H)

1.4211343986592313

## Master equations solutions
In the following we compare our numerical solutions (obtained via `mesolve`) with the analytical solutions of the article. First we consider the state of the battery and of the charger as harmonic oscillators, and then as qubits. 
We are going to compare the plots of the mean energy $E_B(\tau)$ and of the ergotropy $\epsilon_B(\tau)$, defined as:

$$
E_B(\tau) \equiv tr[H_B \rho_B(\tau)] \\
\epsilon_B(\tau) \equiv tr[H_B \rho_B(\tau)] - \min_{U_B} tr[H_B U_B \rho_B(\tau) U_B\dagger ]
$$

In [5]:
# First case, two harmonic oscillators
w_zero = 1.0
a = tensor(destroy(5), qeye(5))
b = tensor(qeye(5), destroy(5))

H_A = w_zero * a.dag() * a
H_B = w_zero * b.dag() * b

F = 1.0
delta_H_A = F * ( exp(-1j*w_zero*t) * a.dag() + exp(1j*w_zero*t) * a)